## Kaggle dataset scraper v2

this script scrapes the description and notebooks page of competitions and datasets  
boot time of the browser has a big impact on overall runtime  
we improve the selenium scraper to batch download websites  

In [1]:
# imports
import requests
import json
import os
import time
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

In [8]:
# let's speed up the scraping proccess by scraping multiple urls in one batch

# generic url scraper

def url_scraper_batch(
                urlbase = '',
                urls = [],
                selenium = True,
                windowSize = "1280,720",
                headless = True,
                quitOnEnd = True,
                waitForElement = '',
                waitForId = '',
                waitForClass = '',
                waitForIframeById = '',
                pressLink = [],
                waitBetweenPress = 30,
                waitUntilTimeout = 20,
               ):
    #meta = {
    #    'url': url
    #}
    
    # simple version
    if selenium == False:
        #page = requests.get(url)
        #meta['status'] = page.status_code
        #html = page.content
        print('batched url is for selenium only')
        return ''
      
    # selenium version
    if selenium == True:
        CHROMEDRIVER_PATH = './.selenium/chromedriver'
        WINDOW_SIZE = windowSize

        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
        chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)

        driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, options=chrome_options)
        driver.implicitly_wait(waitBetweenPress)
        
        result = []
        for url in urls:
            try:
                item = {
                    'url': url
                }

                driver.get(urlbase + url)

                # wait for a given tag to be loaded (javascript generated code)
                if waitForElement != '':
                    try:
                        element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.TAG_NAME, waitForElement)))
                        print ("Page is ready!", urlbase + url)
                    except TimeoutException:
                        print ("element - loading took too much time!", urlbase + url)

                # wait for a given ID to be loaded (javascript generated code)
                if waitForId != '':
                    try:
                        element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.ID, waitForId)))
                        print ("Page is ready!", urlbase + url)
                    except TimeoutException:
                        print ("id - loading took too much time!", urlbase + url)

                # wait for a given class to be loaded (javascript generated code)
                if waitForClass != '':
                    try:
                        # not-found
                        element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.CLASS_NAME, waitForClass)))
                        print ("Page is ready!", urlbase + url)
                    except TimeoutException:
                        print ("class - loading took too much time!", urlbase + url)

                # wait for a given ID to be loaded (javascript generated code)
                if waitForIframeById != '':
                    try:
                        element = WebDriverWait(driver, waitUntilTimeout).until(EC.presence_of_element_located((By.ID, waitForIframeById)))

                        driver.switch_to.frame(element)

                        item['iframe'] = driver.page_source

                        driver.switch_to.default_content()

                        print ("Page is ready!", urlbase + url)
                    except TimeoutException:
                        print ("iframe - loading took too much time!", urlbase + url)

                # press link
                if len(pressLink)>0:
                    driver.implicitly_wait(waitBetweenPress)
                    try:
                        for link in pressLink:
                            print(link)
                            element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.LINK_TEXT, link)))
                            element.click()
                    except TimeoutException:
                        print ("link - loading took too much time!", urlbase + url)

                item['html'] = driver.page_source
                result.append(item)
                
            except Exception as e:
                print("Oops!", e.__class__, "occurred.")
                print(e)
                #break
            
            
        if quitOnEnd:
            driver.close()
    
    return result

urlbase = 'https://www.kaggle.com'
urls = [
    '/austinreese/craigslist-carstrucks-data',
    '/google/tinyquickdraw',
    '/babyoda/women-entrepreneurship-and-labor-force',
]
urls2 = [
    'https://www.kaggle.com/asimandia/lyft3d-inference-kernel/notebook',
    'https://www.kaggle.com/opanichev/lightgbm-and-simple-features',
]

#result = url_scraper_batch(urlbase, urls, waitForClass='content-box', headless = False, quitOnEnd = True)
result = url_scraper_batch('', urls2, waitForIframeById='rendered-kernel-content', headless = False, quitOnEnd = True)
#print(result)
for item in result:
    print(item['url'], len(item['html']))
    #print(len(item['iframe']))

Page is ready! https://www.kaggle.com/asimandia/lyft3d-inference-kernel/notebook
iframe - loading took too much time! https://www.kaggle.com/opanichev/lightgbm-and-simple-features
https://www.kaggle.com/asimandia/lyft3d-inference-kernel/notebook 465884
https://www.kaggle.com/opanichev/lightgbm-and-simple-features 3222390


In [3]:
# generic store data to file function
def store_data(data, file, mode='w', toJson=False):
    if toJson:
        data = json.dumps(data)
    with open(file, mode, encoding='utf-8') as fp:
        result = fp.write(data)
        return result
    
# generic load data from file function
def load_data(file, fromJson=False):
    if os.path.isfile(file):
        with open(file, 'r', encoding='utf-8', errors="ignore") as fp:
            data = fp.read()
            if fromJson:
                data = json.loads(data)
            return data
    else:
        return 'file not found'

# test text
#print(store_data('Hello', '../data/repositories/mlart/test.txt'))
#print(load_data('../data/repositories/mlart/test.txt'))

# test json
#print(store_data({'msg':'Hello World'}, '../data/repositories/mlart/test.json', toJson=True))
#print(load_data('../data/repositories/mlart/test.json', fromJson=True))

#store_data(result[0]['html'], '../data/repositories/kaggle/notebook.html')
#store_data(result[0]['iframe'], '../data/repositories/kaggle/kernel.html')

5
Hello
22
{'msg': 'Hello World'}


In [4]:
# helper function to create folder create_folder
def create_folder(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
            print(path + ' created')
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [ ]:
# get all sites from csv and store them for later feature scraping

# competitions
#csv = '../data/repositories/kaggle/kaggle_competitions.csv'
#folder = '../data/repositories/kaggle/competitions'

# datasets
csv = '../data/repositories/kaggle/kaggle_datasets.csv'
folder = '../data/repositories/kaggle/datasets'

df = pd.read_csv(csv)
df_links = list(df['link'])
batch_size = 25

chunks = [df_links[i:i + batch_size] for i in range(0, len(df_links), batch_size)]
chunks_len = len(chunks)
print('total/chunks', len(df_links), len(chunks))

for i, chunk in enumerate(chunks):
    print('### chunk', i, '/', chunks_len, '###')
    
    # check if chunk is already scraped
    chunk_datasets = []
    chunk_notebooks = []
    for j, link in enumerate(chunk):
        print('# chunk', i, '/', chunks_len, '# item ', j, link)

        temp = link.split('/')
        author = temp[1]
        name = temp[2]
        name = name.replace(' ','_').replace(':','')
        #print(author, name)
    
        # check datasets
        file_d = os.path.join(folder, author, name+'/dataset.html')
        if not os.path.isfile(file_d):
            #print('dataset.html not found', file)
            chunk_datasets.append(link)
            
            # create folders
            create_folder(os.path.join(folder, author+'/'))
            create_folder(os.path.join(folder, author, name+'/'))
        #else:
        #    print('dataset.html already exists')
            
        # check if notebooks.html exists
        file_n = os.path.join(folder, author, name+'/notebooks.html')
        if not os.path.isfile(file_n):
            
            # before adding the link, check if there are notebooks attached
            html = load_data(file_d)
            soup = BeautifulSoup(html, 'html.parser')
            notebooks = soup.select('span.pageheader__link-count > span')
            if len(notebooks) > 0:
            
                #print('notebooks.html not found', file)
                chunk_notebooks.append(link+'/notebooks?sortBy=voteCount')

        
    
    #print(len(chunk_output), chunk_output)
    
    # get content for chunk (datasets)
    if len(chunk_datasets) > 0:
        result = url_scraper_batch(urlbase, chunk_datasets, waitForClass='content-box', headless = True, quitOnEnd = True)
    
        for item in result:
            print(item['url'], len(item['html']))
            #file = os.path.join(folder, item['url']) + '/dataset.html'
            file = folder + item['url'] + '/dataset.html'
            file = file.replace(' ','_').replace(':','')
            print(file)
            store_data(item['html'], file)
    
    # get content for chunk (notebooks)
    if len(chunk_notebooks) > 0:
        result = url_scraper_batch(urlbase, chunk_notebooks, waitForClass='sc-qcrOD', headless = True, quitOnEnd = True)
        
        for item in result:
            print(item['url'], len(item['html']))
            #file = os.path.join(folder, item['url']) + '/notebooks.html'
            file = folder + item['url'] + '.html'
            file = file.replace(' ','_').replace(':','').replace('?sortBy=voteCount','')
            print(file)
            store_data(item['html'], file)
    
        #break
    



In [ ]:
# scrape links from notebooks.html

folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/'
file = 'notebooks.html'
out = 'notebooks.json'
url = 'https://www.kaggle.com/c/3d-object-detection-for-autonomous-vehicles/notebooks?sortBy=voteCount'

def scrape_notebook_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    result = []
    
    partial = soup.find('div', class_="km-list km-list--avatar-list km-list--three-line")
    if partial == None:
        return []
    items = partial.find_all('li', {"role": "listitem"})
    #print(len(items))
    
    for i, item in enumerate(items):
        #print(i, item)
        try:
            link = item.select('a.sc-qcrOD')[0].get('href')
            result.append(link)
        except:
            print('an error occured')
            break
        
        #print(link)
    
    return result

html = load_data(folder_base+folder+file)
#result = url_scraper_batch('', [url], waitForClass='sc-qcrOD', headless = False, quitOnEnd = True)
#html = result[0]['html']
links = scrape_notebook_links(html)
print(links)
#store_data(links, folder_base+folder+out, toJson=True)

In [ ]:
# parse notebooks.html to notebooks.json (links)

#path = '../data/repositories/kaggle/competitions/'
path = '../data/repositories/kaggle/datasets/'
file = 'notebooks.html'
folders = os.listdir(path)
for folder in folders:
    subfolders = os.listdir(os.path.join(path,folder))
    for subfolder in subfolders:
        item = os.path.join(path,folder,subfolder,file)
        print(item)
        if os.path.isfile(item):
            html = load_data(item)
            links = scrape_notebook_links(html)
            print('link', links)
            if len(links) > 0 and not 'titanic' in link.lower():
                output = item.replace('.html','.json')
                print('output', output)
                store_data(links, output, toJson=True)

In [11]:
# download notebooks

#path = '../data/repositories/kaggle/competitions/'
path = '../data/repositories/kaggle/datasets/'
file = 'notebooks.json'
storefolder = 'notebooks/'
urlbase = 'https://www.kaggle.com'
notebook = 'notebook_02.html'
kernel = 'kernel.html'

folders = os.listdir(path)
print(len(folders))
i = 0

for folder in folders:
    subfolders = os.listdir(os.path.join(path,folder))
    for subfolder in subfolders:
        item = os.path.join(path,folder,subfolder,file)
        i+=1

        print('###', i, item)
        if os.path.isfile(item):
            links = load_data(item, fromJson=True)
            #print(len(links), links)

            chunk = []
            for j, link in enumerate(links):
                #print(j, link)

                temp = link.split('/')
                author = temp[1]
                name = temp[2]
                name = name.replace(' ','_').replace(':','')

                # create folders
                f1 = os.path.join(path, folder, subfolder, storefolder, author+'/')
                f2 = os.path.join(path, folder, subfolder, storefolder, author, name+'/')
                create_folder(f1)
                create_folder(f2)

                # check notebooks
                fp = os.path.join(f2, notebook)
                #print(fp)
                #break
                if not os.path.isfile(fp) and not 'kerneler' in fp:
                    #print('notebooks.html not found', fp)
                    chunk.append(link)

            print('### chunk:', i, 'len:', len(chunk), 'links:', chunk)

            # get content for chunk (datasets)
            if len(chunk) > 0:
                result = []
                
                # try fetching data
                try:
                    result = url_scraper_batch(urlbase, chunk, waitForIframeById='rendered-kernel-content', headless = True)
                    
                # catch exception
                except Exception as e:
                    print("Oops!", e.__class__, "occurred.")
                    print(e)
                    #break
                
                # finally store already fetched data
                finally:
                    for item in result:
                        # store notebook
                        #print(item['url'], len(item['html']))
                        fp = path + '/' + folder + '/' + subfolder + '/' + storefolder + '/' + item['url'] + '/' + notebook
                        fp = fp.replace(' ','_').replace(':','').replace('//','/').replace('//','/')
                        print('### output file', fp)
                        store_data(item['html'], fp)
                        
                        # store kernel
                        if 'iframe' in item:
                            fp = fp.replace(notebook, kernel)
                            print('### output file', fp)
                            store_data(item['iframe'], fp)
                   
            quit = 100000
            if i > quit:
                break


8467
### 1 ../data/repositories/kaggle/datasets/4quant\depth-generation-lightfield-imaging\notebooks.json
### chunk: 1 len: 0 links: []
### 2 ../data/repositories/kaggle/datasets/4quant\eye-gaze\notebooks.json
### chunk: 2 len: 0 links: []
### 3 ../data/repositories/kaggle/datasets/4quant\simplefoam\notebooks.json
### chunk: 3 len: 0 links: []
### 4 ../data/repositories/kaggle/datasets/4quant\soft-tissue-sarcoma\notebooks.json
### chunk: 4 len: 0 links: []
### 5 ../data/repositories/kaggle/datasets/a11rand0m\catdoginfo\notebooks.json
### chunk: 5 len: 0 links: []
### 6 ../data/repositories/kaggle/datasets/a13x10\basic-arabic-vocal-emotions-dataset\notebooks.json
### chunk: 6 len: 0 links: []
### 7 ../data/repositories/kaggle/datasets/a1997425\roberta\notebooks.json
### chunk: 7 len: 0 links: []
### 8 ../data/repositories/kaggle/datasets/a2015003713\militaryaircraftdetectiondataset\notebooks.json
### 9 ../data/repositories/kaggle/datasets/a24998667\bert001\notebooks.json
### chunk: 9 le

### 76 ../data/repositories/kaggle/datasets/abcsds\highest-mountains\notebooks.json
### 77 ../data/repositories/kaggle/datasets/abcsds\pokemon\notebooks.json
### chunk: 77 len: 0 links: []
### 78 ../data/repositories/kaggle/datasets/abcsds\pokemongo\notebooks.json
### chunk: 78 len: 0 links: []
### 79 ../data/repositories/kaggle/datasets/abdallahalidev\plantvillage-dataset\notebooks.json
### chunk: 79 len: 0 links: []
### 80 ../data/repositories/kaggle/datasets/abdelaliezzyn\custom-coco\notebooks.json
### chunk: 80 len: 0 links: []
### 81 ../data/repositories/kaggle/datasets/abdelaliezzyn\humanscoco\notebooks.json
### chunk: 81 len: 0 links: []
### 82 ../data/repositories/kaggle/datasets/abdelghanibelgaid\covid19-morocco\notebooks.json
### chunk: 82 len: 0 links: []
### 83 ../data/repositories/kaggle/datasets/abdelhadik\algerian-provinces-by-population\notebooks.json
### 84 ../data/repositories/kaggle/datasets/abdelhaks\covid19-morocco\notebooks.json
### chunk: 84 len: 0 links: []
### 

### chunk: 172 len: 0 links: []
### 173 ../data/repositories/kaggle/datasets/abhishek\sacremoses\notebooks.json
### chunk: 173 len: 0 links: []
### 174 ../data/repositories/kaggle/datasets/abhishek\siim-dicom-images\notebooks.json
### chunk: 174 len: 0 links: []
### 175 ../data/repositories/kaggle/datasets/abhishek\siim-png-images\notebooks.json
### chunk: 175 len: 0 links: []
### 176 ../data/repositories/kaggle/datasets/abhishek\spooky\notebooks.json
### chunk: 176 len: 0 links: []
### 177 ../data/repositories/kaggle/datasets/abhishek\tez-lib\notebooks.json
### chunk: 177 len: 0 links: []
### 178 ../data/repositories/kaggle/datasets/abhishek\transformers\notebooks.json
### chunk: 178 len: 0 links: []
### 179 ../data/repositories/kaggle/datasets/abhishek\tweet-train-folds-v2\notebooks.json
### chunk: 179 len: 0 links: []
### 180 ../data/repositories/kaggle/datasets/abhishek4273\helmet-dataset\notebooks.json
### chunk: 180 len: 0 links: []
### 181 ../data/repositories/kaggle/datasets/ab

### chunk: 269 len: 0 links: []
### 270 ../data/repositories/kaggle/datasets/adilmar\twitter-pt\notebooks.json
### chunk: 270 len: 0 links: []
### 271 ../data/repositories/kaggle/datasets/adinishad\covid-19-newdata-from-worldometer-7102020\notebooks.json
### 272 ../data/repositories/kaggle/datasets/adiquant\salaries-of-us-presidents-and-vice-presidents\notebooks.json
### chunk: 272 len: 0 links: []
### 273 ../data/repositories/kaggle/datasets/adiquant\white-house-staff-salaries-20172020\notebooks.json
### chunk: 273 len: 0 links: []
### 274 ../data/repositories/kaggle/datasets/adiravin\githubcovid\notebooks.json
### chunk: 274 len: 0 links: []
### 275 ../data/repositories/kaggle/datasets/adithyarganesh\english-premier-league-player-data-20172018\notebooks.json
### chunk: 275 len: 0 links: []
### 276 ../data/repositories/kaggle/datasets/adithyarganesh\english-premier-league-player-data-20182019\notebooks.json
### chunk: 276 len: 0 links: []
### 277 ../data/repositories/kaggle/datasets/a

### 371 ../data/repositories/kaggle/datasets/afrniomelo\pronto-benchmark\notebooks.json
### 372 ../data/repositories/kaggle/datasets/afsaja\workout-supplements-and-nutrition-products\notebooks.json
### chunk: 372 len: 0 links: []
### 373 ../data/repositories/kaggle/datasets/afshannabi\human-coding-and-noncoding-rna-sequences\notebooks.json
### 374 ../data/repositories/kaggle/datasets/afumetto\3dprinter\notebooks.json
### chunk: 374 len: 0 links: []
### 375 ../data/repositories/kaggle/datasets/afzale\cryptocurrency-prices\notebooks.json
### 376 ../data/repositories/kaggle/datasets/afzale\rating-vs-gross-collector\notebooks.json
### chunk: 376 len: 0 links: []
### 377 ../data/repositories/kaggle/datasets/agailloty\house-price-in-canada\notebooks.json
### chunk: 377 len: 0 links: []
### 378 ../data/repositories/kaggle/datasets/agajorte\detroit-daily-temperatures-with-artificial-warming\notebooks.json
### chunk: 378 len: 0 links: []
### 379 ../data/repositories/kaggle/datasets/agajorte\zoo

...

### chunk: 16236 len: 0 links: []
### 16237 ../data/repositories/kaggle/datasets/xwolf12\malicious-and-benign-websites\notebooks.json
### chunk: 16237 len: 0 links: []
### 16238 ../data/repositories/kaggle/datasets/xwolf12\network-traffic-android-malware\notebooks.json
### chunk: 16238 len: 0 links: []
### 16239 ../data/repositories/kaggle/datasets/xxc025\segmentation\notebooks.json
### chunk: 16239 len: 0 links: []
### 16240 ../data/repositories/kaggle/datasets/xybervenom\steam-sale\notebooks.json
### chunk: 16240 len: 0 links: []
### 16241 ../data/repositories/kaggle/datasets/xzha4350\australia-saturday-xlotto-jackpot-numbers\notebooks.json
### 16242 ../data/repositories/kaggle/datasets/yagizcandegirmenci\turkey-covid-19-complete-dataset\notebooks.json
### 16243 ../data/repositories/kaggle/datasets/yagunnersya\fifa-21-messy-raw-dataset-for-cleaning-exploring\notebooks.json
### 16244 ../data/repositories/kaggle/datasets/yakinrubaiat\bangladesh-weather-dataset\notebooks.json
### chunk:

### 16310 ../data/repositories/kaggle/datasets/yesteapea\program-params-demo-config\notebooks.json
### chunk: 16310 len: 0 links: []
### 16311 ../data/repositories/kaggle/datasets/yetman\english-contractions\notebooks.json
### chunk: 16311 len: 0 links: []
### 16312 ../data/repositories/kaggle/datasets/yetman\finetuned-bert-for-jigsaw-toxicity-classification\notebooks.json
### chunk: 16312 len: 0 links: []
### 16313 ../data/repositories/kaggle/datasets/yf2zhang\ufo-sight\notebooks.json
### 16314 ../data/repositories/kaggle/datasets/yfyangd\pandastutorials\notebooks.json
### chunk: 16314 len: 0 links: []
### 16315 ../data/repositories/kaggle/datasets/yggrson\insolvencies-in-uk-from-1984-until-2020\notebooks.json
### chunk: 16315 len: 0 links: []
### 16316 ../data/repositories/kaggle/datasets/yi8306\可見光與無線電波的觀測資料之交叉配對\notebooks.json
### 16317 ../data/repositories/kaggle/datasets/yi8306\天文物理_可見光望遠鏡與無線電波望遠鏡之觀測資料交叉配對\notebooks.json
### chunk: 16317 len: 0 links: []
### 16318 ../data/reposit

### chunk: 16396 len: 0 links: []
### 16397 ../data/repositories/kaggle/datasets/yufuin\pytorch-albert-base-v2\notebooks.json
### chunk: 16397 len: 0 links: []
### 16398 ../data/repositories/kaggle/datasets/yugandhar123\isoalpha2codes\notebooks.json
### chunk: 16398 len: 0 links: []
### 16399 ../data/repositories/kaggle/datasets/yugandhar123\openminedslackchannellocationdata\notebooks.json
### chunk: 16399 len: 0 links: []
### 16400 ../data/repositories/kaggle/datasets/yuhaowang\moviesentimentanalysis\notebooks.json
### 16401 ../data/repositories/kaggle/datasets/yuhongc\gbt-energy-detection-outputs-fine-resolution\notebooks.json
### chunk: 16401 len: 0 links: []
### 16402 ../data/repositories/kaggle/datasets/yuhongc\gbt-observation-spectogram-fine-resolution\notebooks.json
### 16403 ../data/repositories/kaggle/datasets/yuhuaxiong\grocery\notebooks.json
### 16404 ../data/repositories/kaggle/datasets/yuiikin\vietnamese-vnexpress-news\notebooks.json
### chunk: 16404 len: 0 links: []
### 1

### 16475 ../data/repositories/kaggle/datasets/zaraks\pascal-voc-2007\notebooks.json
### chunk: 16475 len: 0 links: []
### 16476 ../data/repositories/kaggle/datasets/zarinhelena\chemicals-in-cosmetics\notebooks.json
### chunk: 16476 len: 0 links: []
### 16477 ../data/repositories/kaggle/datasets/zarkonium\synthetic-image-dataset-cats-dogs-bikes-cars\notebooks.json
### chunk: 16477 len: 0 links: []
### 16478 ../data/repositories/kaggle/datasets/zaurbegiev\my-dataset\notebooks.json
### chunk: 16478 len: 0 links: []
### 16479 ../data/repositories/kaggle/datasets/zaynena\selfdriving-car-simulator\notebooks.json
### chunk: 16479 len: 0 links: []
### 16480 ../data/repositories/kaggle/datasets/zazueta\pisa-scores-2015\notebooks.json
### chunk: 16480 len: 0 links: []
### 16481 ../data/repositories/kaggle/datasets/ze1598\animelivechart\notebooks.json
### 16482 ../data/repositories/kaggle/datasets/zealjeung\philippine-stock-exchange-ohlc-data-for-2018\notebooks.json
### chunk: 16482 len: 0 links

### chunk: 16554 len: 0 links: []
### 16555 ../data/repositories/kaggle/datasets/zohebabai\world-bank-data-of-indian-economy-since-1991\notebooks.json
### chunk: 16555 len: 0 links: []
### 16556 ../data/repositories/kaggle/datasets/zohrarezgui\coronavirus-clinical-trials-dataset\notebooks.json
### chunk: 16556 len: 0 links: []
### 16557 ../data/repositories/kaggle/datasets/zollkron\spain-covid19-official-data-from-health-ministry\notebooks.json
### chunk: 16557 len: 0 links: []
### 16558 ../data/repositories/kaggle/datasets/zonnalobo\daily-electricity-usage\notebooks.json
### chunk: 16558 len: 0 links: []
### 16559 ../data/repositories/kaggle/datasets/zqwzoo\yolov5\notebooks.json
### chunk: 16559 len: 0 links: []
### 16560 ../data/repositories/kaggle/datasets/zshujon\40k-bangla-newspaper-article\notebooks.json
### chunk: 16560 len: 0 links: []
### 16561 ../data/repositories/kaggle/datasets/ztaihong\fruits360v14\notebooks.json
### chunk: 16561 len: 0 links: []
### 16562 ../data/reposito